# Car Number Plate Detector

In [1]:
#importing nessesary liberies
import cv2
import imutils
import numpy as np
import pytesseract
from PIL import Image



In [2]:
#image preprocessing
img = cv2.imread('4.jpg',cv2.IMREAD_COLOR)

img = cv2.resize(img, (620,480) )

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #convert to grey scale
gray = cv2.bilateralFilter(gray, 11, 17, 17) #Blur to reduce noise
edged = cv2.Canny(gray, 30, 200) #Perform Edge detection



In [3]:
cv2.imshow('image',gray)
cv2.imshow('Cropped',edged)


cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
# find contours in the edged image, keep only the largest

cnts = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:10]
screenCnt = None

# loop over our contours
for c in cnts:
    # approximate the contour
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.018 * peri, True)

    # if our approximated contour has four points, then
    # we can assume that we have found our screen
    if len(approx) == 4:
        screenCnt = approx
        break
if screenCnt is None:
        detected = 0
        print("No contour detected")
else:
        detected = 1

if detected == 1:
        cv2.drawContours(img, [screenCnt], -1, (0, 255, 0), 3)

# Masking the part other than the number plate
mask = np.zeros(gray.shape,np.uint8)
new_image = cv2.drawContours(mask,[screenCnt],0,255,-1,)
new_image = cv2.bitwise_and(img,img,mask=mask)



In [6]:
cv2.imshow('image',new_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
# Now crop
(x, y) = np.where(mask == 255)
(topx, topy) = (np.min(x), np.min(y))
(bottomx, bottomy) = (np.max(x), np.max(y))
Cropped = gray[topx:bottomx+1, topy:bottomy+1]

In [8]:
#Read the number plate

#pytesseract.pytesseract.tesseract_cmd = r'C:\Users\USER\AppData\Local\Tesseract-OCR\tesseract.exe'
#text = pytesseract.image_to_string(Cropped, config='--psm 11')

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
text = pytesseract.image_to_string(Cropped)
print("Detected Number is:",text)

cv2.imshow('image',img)
cv2.imshow('Cropped',Cropped)

cv2.waitKey(0)
cv2.destroyAllWindows()

Detected Number is: PHR. 26.BR 9044;


In [ ]:
detected

## Main app

In [ ]:
# defining function
import cv2
import imutils
import numpy as np
import pytesseract
from PIL import Image
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

threshold_area = 1800     #threshold area 

def imgprocess(img):
    #img = cv2.imread('4.jpg',cv2.IMREAD_COLOR)

    img = cv2.resize(img, (620,480) )

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #convert to grey scale
    gray = cv2.bilateralFilter(gray, 11, 17, 17) #Blur to reduce noise
    edged = cv2.Canny(gray, 30, 200) #Perform Edge detection

    # find contours in the edged image, keep only the largest
    # ones, and initialize our screen contour
    cnts = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:3]
    screenCnt = None
    # loop over our contours
    for c in cnts:
        area = cv2.contourArea(c)
        #print(area)
        if area > threshold_area and area < threshold_area+1000:
            # approximate the contour
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.018 * peri, True)

            # if our approximated contour has four points, then
            # we can assume that we have found our screen
            if len(approx) == 4:
                x,y,w,h = cv2.boundingRect(c)
                #print(w,h,x,y)
                if abs(w-h) > 3 and w>h :
                    #print(w,h,x,y)
                    screenCnt = approx
                    break
    if screenCnt is None:
            detected = 0
            #print("No contour detected")
    else:
            detected = 1

    if detected == 1:
            cv2.drawContours(img, [screenCnt], -1, (0, 255, 0), 3)

            # Masking the part other than the number plate
            mask = np.zeros(gray.shape,np.uint8)
            new_image = cv2.drawContours(mask,[screenCnt],0,255,-1,)
            #new_image = cv2.bitwise_and(img,img,mask=mask)
            new_image = cv2.bitwise_and(edged,edged,mask=mask)
#edged
            # Now crop
            (x, y) = np.where(mask == 255)
            (topx, topy) = (np.min(x), np.min(y))
            (bottomx, bottomy) = (np.max(x), np.max(y))
            Cropped = gray[topx:bottomx+1, topy:bottomy+1]
            text = pytesseract.image_to_string(Cropped)
            print(text)
    return edged


In [ ]:
# Load video stream
cap = cv2.VideoCapture("car.mp4")


# Get first frame
ret, first_frame = cap.read()
# previous_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
# hsv = np.zeros_like(first_frame)
# hsv[...,1] = 255
while True:
    # Read of video file
    ret, frame = cap.read()
    img=imgprocess(frame)
    cv2.imshow('Number plate detection', img)
    
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
    
    
    #text = pytesseract.image_to_string(final)
    #print("Detected Number is:",text)
    
cap.release()
cv2.destroyAllWindows()